In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

In [2]:
import logging


logging.basicConfig(
    level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s'
)

### Prepare input file

In [3]:
from decouple import config
from huggingface_hub import AsyncInferenceClient


HUGGINGFACE_TOKEN = config('HUGGINGFACE_TOKEN')

MODEL_HUB_ID = 'microsoft/Phi-3-mini-4k-instruct'

client = AsyncInferenceClient(
    model=MODEL_HUB_ID,
    provider='hf-inference',
    timeout=None,
    api_key=HUGGINGFACE_TOKEN,
)

In [3]:
from math_rag.application.base.assistants import BaseAssistantInput, BaseAssistantOutput


class SomeInput(BaseAssistantInput):
    pass


class SomeOutput(BaseAssistantOutput):
    result: int

In [4]:
import json

from pathlib import Path

from math_rag.application.models.inference import (
    LLMBatchRequest,
    LLMBatchResult,
    LLMConversation,
    LLMMessage,
    LLMParams,
    LLMRequest,
)
from math_rag.infrastructure.mappings.inference.huggingface import (
    LLMRequestMapping,
    LLMResponseListMapping,
)


MODEL_HUB_ID = 'microsoft/Phi-3-mini-4k-instruct'
some_input = SomeInput()

# request = LLMRequest(
#     conversation=LLMConversation(
#         messages=[
#             LLMMessage(role='system', content='You are a helpful assistant.'),
#             LLMMessage(role='user', content='what is 2+2'),
#         ]
#     ),
#     params=LLMParams(
#         model=MODEL_HUB_ID,
#         temperature=0,
#         response_type=SomeOutput,
#         max_completion_tokens=10,
#         metadata={'input_id': str(some_input.id)},
#     ),
# )

# batch_request: LLMBatchRequest = LLMBatchRequest(requests=[request])

_requests = [
    LLMRequest(
        conversation=LLMConversation(
            messages=[
                LLMMessage(role='system', content='You are a helpful assistant.'),
                LLMMessage(role='user', content=f'what is {i}+2'),
            ]
        ),
        params=LLMParams(
            model=MODEL_HUB_ID,
            temperature=0,
            response_type=SomeOutput,
            max_completion_tokens=10,
            metadata={'input_id': str(some_input.id)},
        ),
    )
    for i in range(200)
]

batch_request: LLMBatchRequest = LLMBatchRequest(requests=_requests)

In [5]:
requests = [LLMRequestMapping.to_target(request) for request in batch_request.requests]
lines = [json.dumps(request, separators=(',', ':')) for request in requests]
jsonl_str = '\n'.join(lines)
jsonl_bytes = jsonl_str.encode('utf-8')

In [7]:
request_dict = json.loads(lines[0])

In [8]:
result = await client.chat_completion(**request_dict)
result

ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='{ "result": 4 }', tool_call_id=None, tool_calls=None), logprobs=None)], created=1744125157, id='', model='microsoft/Phi-3-mini-4k-instruct', system_fingerprint='3.2.1-native', usage=ChatCompletionOutputUsage(completion_tokens=9, prompt_tokens=17, total_tokens=26), object='chat.completion')

In [40]:
response_list = LLMResponseListMapping.to_source(
    result,
    request_id=request.id,
    input_id=request_dict['extra_body']['input_id'],
    response_type=SomeOutput,
)
response_list

LLMResponseList(id=UUID('f7fd3b29-2772-468d-831f-b160bf1dfaa0'), request_id=UUID('cb02d5a3-bed8-4109-b06c-04d585081ef3'), responses=[LLMResponse(id=UUID('3976268b-adbf-494a-b88b-fda24781d477'), content=BoundAssistantOutput(id=UUID('bd7fdedd-f32a-4d7f-845d-0a21413828b6'), input_id=UUID('c882baa3-2047-4f65-b8df-bd176bba3b2c'), result=4), logprobs=None)])

In [ ]:
input_file_path = Path(f'.tmp/input_{batch_request.id}.jsonl')

with open(input_file_path, 'w') as input_file:
    for line in lines:
        input_file.write(line + '\n')

### Apptainer

In [5]:
from math_rag.infrastructure.containers import InfrastructureContainer


infrastructure_container = InfrastructureContainer()
infrastructure_container.init_resources()

tgi_batch_llm = infrastructure_container.tgi_batch_llm()

In [6]:
await tgi_batch_llm.init_resources()

2025-05-07 07:45:25,271 - INFO - Host canonicalization disabled
2025-05-07 07:45:25,272 - INFO - Opening SSH connection to login-gpu.hpc.srce.hr, port 22
2025-05-07 07:45:25,290 - INFO - [conn=0] Connected to SSH server at login-gpu.hpc.srce.hr, port 22
2025-05-07 07:45:25,290 - INFO - [conn=0]   Local address: 172.18.0.7, port 46008
2025-05-07 07:45:25,290 - INFO - [conn=0]   Peer address: 161.53.2.37, port 22
2025-05-07 07:45:25,332 - INFO - [conn=0] Beginning auth for user lpanic
2025-05-07 07:45:25,663 - INFO - [conn=0] Auth for user lpanic succeeded
2025-05-07 07:45:25,666 - INFO - [conn=0, chan=0] Requesting new SSH session
2025-05-07 07:45:25,677 - INFO - [conn=0, chan=0]   Command: mkdir -p tgi_default_root
2025-05-07 07:45:25,834 - INFO - [conn=0, chan=0] Received exit status 0
2025-05-07 07:45:25,839 - INFO - [conn=0, chan=0] Received channel close
2025-05-07 07:45:25,841 - INFO - [conn=0, chan=0] Channel closed
2025-05-07 07:45:25,842 - INFO - Command `mkdir -p tgi_default_r

In [7]:
res = await tgi_batch_llm.batch_generate(
    batch_request=batch_request,
    response_type=SomeOutput,
    poll_interval=3 * 60,
    max_tokens_per_day=None,
    max_num_retries=0,
)
res

2025-05-07 07:53:38,759 - INFO - Host canonicalization disabled
2025-05-07 07:53:38,759 - INFO - Opening SSH connection to login-gpu.hpc.srce.hr, port 22
2025-05-07 07:53:38,769 - INFO - [conn=23] Connected to SSH server at login-gpu.hpc.srce.hr, port 22
2025-05-07 07:53:38,770 - INFO - [conn=23]   Local address: 172.18.0.7, port 45856
2025-05-07 07:53:38,770 - INFO - [conn=23]   Peer address: 161.53.2.37, port 22
2025-05-07 07:53:38,802 - INFO - [conn=23] Beginning auth for user lpanic
2025-05-07 07:53:39,076 - INFO - [conn=23] Auth for user lpanic succeeded
2025-05-07 07:53:39,078 - INFO - [conn=23, chan=0] Requesting new SSH session
2025-05-07 07:53:39,109 - INFO - [conn=23, chan=0]   Subsystem: sftp
2025-05-07 07:53:39,121 - INFO - [conn=23, chan=0] Starting SFTP client
2025-05-07 07:53:39,333 - INFO - [conn=23, chan=0] Received exit status 0
2025-05-07 07:53:39,334 - INFO - [conn=23, chan=0] Received channel close
2025-05-07 07:53:39,336 - INFO - [conn=23, chan=0] SFTP client exit

LLMBatchResult(id=UUID('a3acac71-f213-46c2-8c03-a9436a140e88'), batch_request_id=UUID('07f775dc-2dbb-4d54-a6e8-a384708ad143'), response_lists=[LLMResponseList(id=UUID('c47784c1-49db-4f22-ab45-24fd68d723af'), request_id=UUID('f9781106-4b36-4872-8e12-be4912c59edc'), responses=[LLMResponse(id=UUID('df5d9b39-6f8c-4853-9ee9-d4fa6b589a7f'), content=BoundAssistantOutput(id=UUID('46ae828b-63b4-4edb-9eec-b17fef2a2476'), input_id=UUID('dbf3d0b7-3f3b-4cef-8839-23e5a5d95ef8'), result=83), logprobs=None)]), LLMResponseList(id=UUID('eaf693c8-f8d5-4f23-b1e3-18bef138def7'), request_id=UUID('e5579313-cb49-4533-933e-77b2159e4016'), responses=[LLMResponse(id=UUID('0869f24c-7b09-4a61-960f-3a190e2aad6a'), content=BoundAssistantOutput(id=UUID('191f9913-9025-4e9d-bda8-3a1a5bd49596'), input_id=UUID('dbf3d0b7-3f3b-4cef-8839-23e5a5d95ef8'), result=71), logprobs=None)]), LLMResponseList(id=UUID('66125a4f-3616-4959-97c4-d41ff6af81db'), request_id=UUID('8de4bc2f-29e2-46b3-b74c-0f173554e519'), responses=[LLMRespons

In [9]:
from pathlib import Path


sftp_client = infrastructure_container.sftp_client()

await sftp_client.download(
    Path('tgi_default_root/error.log'), Path('../.tmp/error.log')
)

2025-05-05 09:25:03,554 - INFO - Host canonicalization disabled
2025-05-05 09:25:03,555 - INFO - Opening SSH connection to login-gpu.hpc.srce.hr, port 22
2025-05-05 09:25:03,574 - INFO - [conn=35] Connected to SSH server at login-gpu.hpc.srce.hr, port 22
2025-05-05 09:25:03,575 - INFO - [conn=35]   Local address: 172.18.0.7, port 55530
2025-05-05 09:25:03,575 - INFO - [conn=35]   Peer address: 161.53.2.37, port 22
2025-05-05 09:25:03,613 - INFO - [conn=35] Beginning auth for user lpanic
2025-05-05 09:25:03,879 - INFO - [conn=35] Auth for user lpanic succeeded
2025-05-05 09:25:03,880 - INFO - [conn=35, chan=0] Requesting new SSH session
2025-05-05 09:25:03,903 - INFO - [conn=35, chan=0]   Subsystem: sftp
2025-05-05 09:25:03,911 - INFO - [conn=35, chan=0] Starting SFTP client
2025-05-05 09:25:04,135 - INFO - [conn=35, chan=0] Received exit status 0
2025-05-05 09:25:04,136 - INFO - [conn=35, chan=0] Received channel close
2025-05-05 09:25:04,136 - INFO - [conn=35, chan=0] SFTP client exit

In [8]:
from pathlib import Path


sftp_client = infrastructure_container.sftp_client()

# tar -czvf snapshot.tar.gz data

# TODO download dir
await sftp_client.download(
    Path('tgi_default_root/snapshot.tar.gz'), Path('../.tmp/snapshot.tar.gz')
)

# tar -xzvf snapshot.tar.gz

2025-05-07 08:00:09,206 - INFO - Host canonicalization disabled
2025-05-07 08:00:09,206 - INFO - Opening SSH connection to login-gpu.hpc.srce.hr, port 22
2025-05-07 08:00:09,220 - INFO - [conn=38] Connected to SSH server at login-gpu.hpc.srce.hr, port 22
2025-05-07 08:00:09,221 - INFO - [conn=38]   Local address: 172.18.0.7, port 56070
2025-05-07 08:00:09,221 - INFO - [conn=38]   Peer address: 161.53.2.37, port 22
2025-05-07 08:00:09,257 - INFO - [conn=38] Beginning auth for user lpanic
2025-05-07 08:00:09,528 - INFO - [conn=38] Auth for user lpanic succeeded
2025-05-07 08:00:09,530 - INFO - [conn=38, chan=0] Requesting new SSH session
2025-05-07 08:00:09,561 - INFO - [conn=38, chan=0]   Subsystem: sftp
2025-05-07 08:00:09,570 - INFO - [conn=38, chan=0] Starting SFTP client
2025-05-07 08:00:09,816 - INFO - [conn=38, chan=0] Received exit status 0
2025-05-07 08:00:09,816 - INFO - [conn=38, chan=0] Received channel close
2025-05-07 08:00:09,817 - INFO - [conn=38, chan=0] SFTP client exit

In [ ]:
from os import environ

from decouple import config


environ['PBS_O_WORKDIR'] = '../.tmp'
environ['TGI_API_KEY'] = config('HUGGINGFACE_TOKEN')
environ['MODEL_HUB_ID'] = 'microsoft/Phi-3-mini-4k-instruct'

%run ../assets/hpc/hf/tgi/tgi_client.py